### Read the script

In [1]:
import os
from ieeg.io import get_data, raw_from_layout, save_derivative

In [2]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

In [3]:
#for subject in subjects:
#    raw = raw_from_layout(layout, subject=subject, preload=True, extension='.edf')
    # do stuff with each subject data

subject = subjects[1]
raw = raw_from_layout(layout, subject=subject, preload=True, extension='.edf')
print(subjects)

FileNotFoundError: Search terms matched more than one file: 
[<BIDSFile filename='C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf'>, <BIDSFile filename='C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_ieeg.edf'>] 
 try adding more search terms

In [ ]:
raw.plot()

### Linear noise filtering
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_clean.html

In [4]:
from ieeg.mt_filter import line_filter
from ieeg.viz.ensemble import figure_compare

In [ ]:
# n_jobs=-1 means take all the processors, n_jobs=-2 means take all - 1 processors
# Keep all the other parameters not changed
# %% filter data
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='700ms', freqs=[60], notch_widths=20)
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='20s', freqs=[60, 120, 180, 240],
            notch_widths=20)

In [ ]:
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "clean")):
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
save_derivative(raw, layout, "clean", True)

### Remove bad channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html


In [4]:
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.multitaper import spectrogram
from ieeg.timefreq.utils import crop_pad
from ieeg.navigate import channel_outlier_marker, crop_empty_data
from ieeg.viz.parula import parula_map
import numpy as np
import mne

In [ ]:
del raw

In [ ]:
raw = raw_from_layout(layout.derivatives['derivatives/clean'], subject=subject, preload=False, desc='clean',extension='.edf')
raw = crop_empty_data(raw)

In [24]:
# plot subject
# !!!!! problems: cannot find the surface file##########
from ieeg.viz.mri import plot_subj, plot_on_average
# plot the subject brain
subjects_dir=os.path.join(LAB_root,'BIDS-1.0_LexicalDecRepDelay\\BIDS')
fig1 = plot_subj(raw, subj_dir=subjects_dir)
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

Using pyvistaqt 3d backend.
Could not estimate rigid Talairach alignment, using identity matrix


FileNotFoundError: File does not exist: "C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\D103\surf\lh.pial"

In [10]:
# Remove EEG channels for D101
# eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']
# Remove EEG channels for D103 (same)
eeg_channels_to_exclude = ['F7', 'F8', 'T5', 'T6', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
                           'T3', 'T4', 'O1', 'O2']
# Problems: !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Channel(s) Fz, Cz, Pz, Fp1, Fp2 not found, nothing dropped. But these channels exist in the sub-D0103_task-LexicalDecRepDelay_channels.tsv!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Remove EEG channels for D107

raw.drop_channels(eeg_channels_to_exclude)
#raw.load_data()

<RawEDF | sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf, 234 x 5050945 (2466.3 s), ~180 kB, data not loaded>

In [11]:
# mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 3)
# Exclude bad channels
raw.drop_channels(raw.info['bads'])

outlier round 1 channels: ['LTAI10']
outlier round 1 channels: ['LTAI10', 'LTMM1']
outlier round 1 channels: ['LTAI10', 'LTMM1', 'LTMM4']
outlier round 1 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5']
outlier round 2 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1']
outlier round 2 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2']
outlier round 2 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2', 'LTAI11']
outlier round 2 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2', 'LTAI11', 'LAI18']
outlier round 2 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2', 'LTAI11', 'LAI18', 'LTP5']
outlier round 2 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2', 'LTAI11', 'LAI18', 'LTP5', 'LTMM2']
outlier round 3 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2', 'LTAI11', 'LAI18', 'LTP5', 'LTMM2', 'LTPS15']
outlier round 3 channels: ['LTAI10', 'LTMM1', 'LTMM4', 'LTPM5', 'LTAS1', 'LTAI2', 'LTAI11', 'LA

<RawEDF | sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf, 214 x 5050945 (2466.3 s), ~165 kB, data not loaded>

In [ ]:
# CAR (common average reference) DONT DO THIS NOW
# raw.set_eeg_reference(ref_channels="average", ch_type='seeg')

In [12]:
raw.plot()

Using qt as 2D backend.


### Remove bad trials
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [ ]:
from ieeg.navigate import channel_outlier_marker, trial_ieeg,outliers_to_nan

In [ ]:
epoch = 'Auditory_stim'
times = [-1.2, 2]
trials = trial_ieeg(raw, epoch, times, preload=True)
#!!!!!!!!!!!!!!!!   What does the preload mean!!!!!!!!!!!!!!!!!!!!!!!
outliers_to_nan(trials, outliers=10)

In [16]:
baseline = 'Cue'
times = [-0.5, 0]
baselines = trial_ieeg(raw, baseline, times, preload=True)
outliers_to_nan(baselines, outliers=10)

Used Annotations descriptions: [np.str_('Auditory_stim'), np.str_('CORRECT'), np.str_('Cue'), np.str_('Delay'), np.str_('ERR_RESP_REP_MIS'), np.str_('Repeat'), np.str_('Speak'), np.str_('Yes/No'), np.str_('banic'), np.str_('baron'), np.str_('berin'), np.str_('bison'), np.str_('boril'), np.str_('cabin'), np.str_('caris'), np.str_('cazel'), np.str_('civic'), np.str_('civil'), np.str_('comet'), np.str_('delin'), np.str_('demon'), np.str_('devim'), np.str_('facet'), np.str_('fatal'), np.str_('focal'), np.str_('galed'), np.str_('galel'), np.str_('gapel'), np.str_('gorel'), np.str_('havel'), np.str_('haven'), np.str_('hazel'), np.str_('herib'), np.str_('janem'), np.str_('japel'), np.str_('jokel'), np.str_('kelic'), np.str_('labin'), np.str_('lamin'), np.str_('latin'), np.str_('legal'), np.str_('libel'), np.str_('likel'), np.str_('limid'), np.str_('lomic'), np.str_('madam'), np.str_('mafil'), np.str_('malim'), np.str_('manic'), np.str_('mason'), np.str_('menon'), np.str_('merit'), np.str_('mi

<Epochs | 336 events (all good), -0.5 – 0 s (baseline off), ~562.5 MB, data loaded,
 np.str_('Cue'): 336>

### Multitaper spectrogram plot

#### Calculate Spectra

In [ ]:
freq = np.arange(10, 200., 6.)
#spectra = spectrogram(raw, freq, 'Auditory_stim', -1.2, 1.2, 'Cue', -0.5, 0,
#                      n_jobs=-3, verbose=10, time_bandwidth=10, n_cycles=freq/2)
spectra = spectrogram(trials, baseline=baselines, freqs=freq, n_jobs=-3, time_bandwidth=10, n_cycles=freq/2)
crop_pad(spectra, "0.5s")
# Problem: why so many channels are blank?

In [23]:
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject))
filename = os.path.join(save_dir, subject,'Auditory-tfr.h5')
mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))
##!!!!!!!!!!!!!! Questions: how to read these saved files !!!!!!!!!!!!!

C:\Users\bl314\AppData\Local\Temp\ipykernel_45808\1256316314.py:6: RuntimeWarning: This filename (C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\Auditor-avg.fif) does not conform to MNE naming conventions. All time-frequency object files should end with .h5 or .hdf5
  spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))


In [20]:
raw.plot()

#### Plot data

In [ ]:
chan_grid(spectra, vlim=(0, 2), cmap=parula_map)